In [1]:
import h5py
import pickle
import time
import numpy as np
import pandas as pd
import wavelets as wl
from scipy import signal
from matplotlib import pyplot as plt

#### Get a dictionary with mice's experiment information and another with the spiking activity in h5py format  

In [2]:
# List of mice, classified by condition
IDs_WT = ['SERT1597'] #, 'SERT1659', 'SERT1678', 'SERT1908', 'SERT1984', 'SERT1985', 'SERT2014']
#IDs_KO = ['SERT1668'] #, 'SERT1665', 'SERT2013', 'SERT2018', 'SERT2024']

# Dictionaries to collect info about the experiment and spiking activity
all_info_WT = {}
all_mice_WT = {}


print('Processing wild-types')
for ID in IDs_WT:
    npys_dir = '/home/maspe/filer/SERT/' + ID + '/npys/'
    spikes_dir = '/home/maspe/filer/SERT/' + ID + '/spikes/results/'
    
    print('Loading ' + ID)
    with open(npys_dir + ID + '.info', 'rb') as f:
        info = pickle.load(f)
    
    # Put the info of this mouse into de info dictionary
    all_info_WT[ID] = info
    channels = info['channels_list']
    
    fit = []
    for channel in channels:
        path = spikes_dir + channel + '.result.hdf5'
    
        # Get the spiking activity from the h5py file
        fit.append(h5py.File(path, 'r'))
        
    print('Loaded ' + spikes_dir)
     
    # Put the spiking activity of this mouse into the spiking dictionary
    all_mice_WT[ID] = fit

    
# Same for KO mice
#print('Processing knock-out')
all_info_KO = {}
all_mice_KO = {}
for ID in IDs_KO:
    npys_dir = '/home/maspe/filer/SERT/' + ID + '/npys/'
    spikes_dir = '/home/maspe/filer/SERT/' + ID + '/spikes/results/'
    
    print('Loading ' + ID)
    with open(npys_dir + ID + '.info', 'rb') as f:
        info = pickle.load(f)

    all_info_KO[ID] = info
    channels = info['channels_list']
    
    fit = []
    for channel in channels:
        path = spikes_dir + channel + '.result.hdf5'
    
        fit.append(h5py.File(path, 'r'))
        
    print('Loaded ' + spikes_dir)
        
    all_mice_KO[ID] = fit

    
# all_mice_WT['SERT1597'][0]['spiketimes']['temp_0'][()].shape

Processing wild-types
Loading SERT1597
Loaded /home/maspe/filer/SERT/SERT1597/spikes/results/


NameError: name 'IDs_KO' is not defined

#### Create one dictionary with the ID of each channell and another with the spike times for all units

In [3]:
all_units_WT = {}
all_channels_WT = {}
for mouse in all_mice_WT.keys():
    n_channels = 32
    units = []
    channels_id = []
    
    iteration = 0
    for channel in range(n_channels):
        for unit in all_mice_WT[mouse][channel]['spiketimes'].keys():
            units.append(all_mice_WT[mouse][channel]['spiketimes'][unit][()]) # Final "[()]" is to import values from h5py 
      
            channels_id.append(all_info_WT[mouse]['channels_locs'][iteration])
        
        iteration += 1
        
            
    all_units_WT[mouse] = units
    all_channels_WT[mouse] = channels_id
        
# print('n units = %i' %len(all_units))


all_units_KO = {}
all_channels_KO = {}
for mouse in all_mice_KO.keys():
    n_channels = 32
    units = []
    channels_id = []
    
    iteration = 0
    for channel in range(n_channels):
        for unit in all_mice_KO[mouse][channel]['spiketimes'].keys():
            units.append(all_mice_KO[mouse][channel]['spiketimes'][unit][()]) # Final "[()]" is to import values from h5py 
      
            channels_id.append(all_info_KO[mouse]['channels_locs'][iteration])
        
        iteration += 1
        
            
    all_units_KO[mouse] = units
    all_channels_KO[mouse] = channels_id
        
# print('n units = %i' %len(all_units))
# all_units_WT['SERT1597']

In [ ]:
47966705 / 30000 / 60

In [ ]:
19382520 / 30000 / 60

In [ ]:
stopHC

In [ ]:
np.int(all_info_WT[mouse]['startOF'])

In [ ]:
stopOF

In [ ]:
17550000 / 30000 / 60.0

#### Create the windows of interest

We are currently using a time windows of 2 minutes that finishes 15 seconds before the beginning of the OF, for the HC condition; and a time windows that begins 15 seconds after the beginning of the OF, and extends to minute 10 (total length: 9.75 min).

In [4]:
### THIS CAN BE MOVED TO THE CREATION OF INFO FILES!!! ###
### Extract a windows of n_secs length at the beginning and end of the task
n_min = 2
exclude_sec = 15
sample_rate = 30000
#window = int(sampleRate * secs)
n_points = 30000 * 60 * 10 # Length of the OF
window = sample_rate * n_min * 60 # 2 min windows
exclude_window = sample_rate * exclude_sec # 15 sec windows to exclude at the HC-OF transition

all_epochs_WT = {}
all_perispikes_WT = {}
for mouse in all_mice_WT.keys():
    stopHC = np.int(all_info_WT[mouse]['startOF']) - exclude_window
    startHC = stopHC - window
    
    startOF = np.int(all_info_WT[mouse]['startOF']) + exclude_window
    stopOF = startOF + n_points - exclude_window
    
    baseline = np.arange(startHC, stopHC, 1)
    task = np.arange(startOF, stopOF, 1)
    
    complete_window = np.concatenate([baseline, task])
    all_epochs_WT[mouse] = complete_window
    

all_epochs_KO = {}
all_perispikes_KO = {}
for mouse in all_mice_KO.keys():
    stopHC = np.int(all_info_KO[mouse]['startOF']) - exclude_window
    startHC = stopHC - window
    
    startOF = np.int(all_info_KO[mouse]['startOF']) + exclude_window
    stopOF = startOF + n_points - exclude_window
    
    baseline = np.arange(startHC, stopHC, 1)
    task = np.arange(startOF, stopOF, 1)
    
    complete_window = np.concatenate([baseline, task])
    all_epochs_KO[mouse] = complete_window


task_npoints = complete_window.shape[0]

#### Extracting the spikes that fall inside the time 

In [5]:
all_perispikes_WT = {}
#npys_dir = '/home/maspe/filer/SERT/ALL/npys/'


for mouse in all_mice_WT.keys():
    clock = time.time()

    all_spikes = all_units_WT[mouse]
    peristimulus_spikes = np.zeros((len(all_spikes), task_npoints))   
    
    print('Processing mouse %s...' % mouse)
    
    for unit in range(len(all_spikes)):
        peristimulus_spikes[unit, :] = np.isin(all_epochs_WT[mouse], all_spikes[unit])#, axes=(1, 0))
#         peristimulus_spikes[:, :, ] = np.transpose(np.isin(epochs_matrix, all_spikes[unit]), axes=(0, 2, 1))
# #         peristimulus_spikes = np.transpose(np.array(peristimulus_spikes), axes=(0, 2, 1))
        
    all_perispikes_WT[mouse] = peristimulus_spikes
    #np.save(npys_dir + mouse + '_perispikes_WT.npy', peristimulus_spikes)
    
print('Done in {} s.!'.format(time.time() - clock)

Processing mouse SERT1597...
Done!


In [7]:
all_perispikes_WT['SERT1597'].shape

(51, 21150000)

In [ ]:
all_perispikes_KO = {}

for mouse in all_mice_KO.keys():
    all_spikes = all_units_KO[mouse] 
    peristimulus_spikes = np.zeros((len(all_spikes), task_npoints))   
    
    print('Processing mouse %s...' % mouse)

    for unit in range(len(all_spikes)):
        peristimulus_spikes[unit, :] = np.isin(all_epochs_KO[mouse], all_spikes[unit])
        
    #np.save(npys_dir + mouse + '_perispikes_KO.npy', peristimulus_spikes)

In [ ]:
all_perispikes_KO.keys()

####################################

#### Loading spikes

In [ ]:
import numpy as np
npys_dir = '/home/maspe/filer/SERT/ALL/npys/'

In [ ]:
all_mice_WT = {'SERT1597': np.load(npys_dir + 'SERT1597_perispikes_WT.npy', allow_pickle=True), 
               'SERT1659': np.load(npys_dir + 'SERT1659_perispikes_WT.npy', allow_pickle=True),
              'SERT1678': np.load(npys_dir + 'SERT1678_perispikes_WT.npy', allow_pickle=True),
              'SERT1908': np.load(npys_dir + 'SERT1908_perispikes_WT.npy', allow_pickle=True),
              'SERT1984': np.load(npys_dir + 'SERT1984_perispikes_WT.npy', allow_pickle=True),
              'SERT1985': np.load(npys_dir + 'SERT1985_perispikes_WT.npy', allow_pickle=True),
              'SERT2014': np.load(npys_dir + 'SERT2014_perispikes_WT.npy', allow_pickle=True)}

print('Perispikes loaded!\nAdding spikes...')

mPFC_WT = {}
NAC_WT  = {}
BLA_WT  = {}
vHip_WT = {}

for mouse in all_mice_WT.keys():
    mPFC_WT_indexes  = [i for i,x in enumerate(all_channels_WT[mouse]) if x == 'mPFC_left']
    NAC_WT_indexes = [i for i,x in enumerate(all_channels_WT[mouse]) if x == 'NAC_left']
    BLA_WT_indexes  = [i for i,x in enumerate(all_channels_WT[mouse]) if x == 'BLA_left']
    vHip_WT_indexes  = [i for i,x in enumerate(all_channels_WT[mouse]) if x == 'vHipp_left']
    
    
    mPFC_WT[mouse] = np.sum(all_mice_WT[mouse][mPFC_WT_indexes, :], axis=0)
    NAC_WT[mouse] = np.sum(all_mice_WT[mouse][NAC_WT_indexes, :], axis=0)
    BLA_WT[mouse] = np.sum(all_mice_WT[mouse][BLA_WT_indexes, :], axis=0)
    vHip_WT[mouse] = np.sum(all_mice_WT[mouse][vHip_WT_indexes, :], axis=0)

In [9]:
len(all_channels_WT[mouse])

51

In [ ]:
npys_dir = '/home/maspe/filer/SERT/ALL/npys/'

all_spikes_WT = {'mPFC': mPFC_WT, 'BLA': BLA_WT, 'NAC': NAC_WT, 'vHip': vHip_WT}
np.save('/home/maspe/filer/SERT/ALL/npys/all_spikes_WT', all_spikes_WT)

In [ ]:
all_channels_KO[mouse]

In [ ]:
npys_dir = '/home/maspe/filer/SERT/ALL/npys/'
all_mice_KO = {'SERT1665': np.load(npys_dir + 'SERT1665_perispikes_KO.npy', allow_pickle=True), 
               'SERT1668': np.load(npys_dir + 'SERT1668_perispikes_KO.npy', allow_pickle=True),
              'SERT2013': np.load(npys_dir + 'SERT2013_perispikes_KO.npy', allow_pickle=True),
              'SERT2018': np.load(npys_dir + 'SERT2018_perispikes_KO.npy', allow_pickle=True),
              'SERT2024': np.load(npys_dir + 'SERT2024_perispikes_KO.npy', allow_pickle=True)}
print('Perispikes loaded!\nAdding spikes...')



In [ ]:
mPFC_KO = {}
NAC_KO  = {}
BLA_KO  = {}
vHip_KO = {}

for mouse in all_mice_KO.keys():
    mPFC_KO_indexes  = [i for i,x in enumerate(all_channels_KO[mouse]) if x == 'mPFC_left']
    NAC_KO_indexes = [i for i,x in enumerate(all_channels_KO[mouse]) if x == 'NAC_left']
    BLA_KO_indexes  = [i for i,x in enumerate(all_channels_KO[mouse]) if x == 'BLA_left']
    vHip_KO_indexes  = [i for i,x in enumerate(all_channels_KO[mouse]) if x == 'vHipp_left']
    
    
    mPFC_KO[mouse] = np.sum(all_mice_KO[mouse][mPFC_KO_indexes, :], axis=0)
    NAC_KO[mouse] = np.sum(all_mice_KO[mouse][NAC_KO_indexes, :], axis=0)
    BLA_KO[mouse] = np.sum(all_mice_KO[mouse][BLA_KO_indexes, :], axis=0)
    vHip_KO[mouse] = np.sum(all_mice_KO[mouse][vHip_KO_indexes, :], axis=0)

In [ ]:
all_spikes_KO = {'mPFC': mPFC_KO, 'BLA': BLA_KO, 'NAC': NAC_KO, 'vHip': vHip_KO}
np.save('/home/maspe/filer/SERT/ALL/npys/all_spikes_KO', all_spikes_KO)

In [ ]:
npys_dir

#######################

In [ ]:
all_spikes_WT = np.load('/home/maspe/filer/SERT/ALL/npys/all_spikes_WT.npy', allow_pickle=True).item()
all_spikes_WT.keys()

In [ ]:
all_spikes_KO = np.load('/home/maspe/filer/SERT/ALL/npys/all_spikes_KO.npy', allow_pickle=True).item()
all_spikes_KO.keys()

In [ ]:
tmean_WT.shape[0]/2

In [ ]:
factor = 15000
structure = 'vHip'

iteration = 0
t = np.zeros((len(all_spikes_WT[structure].keys()), 7200000 / factor))
for mouse in all_spikes_WT[structure].keys():
    t[iteration,:]=np.add.reduceat(all_spikes_WT[structure][mouse],
                                   range(0, all_spikes_WT[structure][mouse].shape[0], factor))
    
    iteration += 1
    
tmean_WT=np.mean(t, axis=0)
tsem_WT=np.std(t, axis=0) / np.sqrt(t.shape[0])

iteration = 0
t = np.zeros((len(all_spikes_KO[structure].keys()), 7200000 / factor))
for mouse in all_spikes_KO[structure].keys():
    t[iteration,:]=np.add.reduceat(all_spikes_KO[structure][mouse],
                                   range(0, all_spikes_KO[structure][mouse].shape[0], factor))
    
    iteration += 1
    
tmean_KO=np.mean(t, axis=0)
tsem_KO=np.std(t, axis=0) / np.sqrt(t.shape[0])

tx = range(0, tmean_KO.shape[0])


### Plot
#plt.figure(figsize=(20,10))
#plt.subplot(2,2,1)

f, (ax, ax2) = plt.subplots(1, 2, sharey=True)
ax.plot(tx, tmean_WT, '-', color='blue', alpha=1)
ax.fill_between(tx, tmean_WT + tsem_WT, tmean_WT - tsem_WT, alpha=0.5)

ax.plot(tx, tmean_KO, '-', color='red', alpha=1)
ax.fill_between(tx, tmean_KO + tsem_KO, tmean_KO - tsem_KO, alpha=0.5)

ax2.plot(tx, tmean_WT, '-', color='blue', alpha=1)
ax2.fill_between(tx, tmean_WT + tsem_WT, tmean_WT - tsem_WT, alpha=0.5)

ax2.plot(tx, tmean_KO, '-', color='red', alpha=1)
ax2.fill_between(tx, tmean_KO + tsem_KO, tmean_KO - tsem_KO, alpha=0.5)

ax.set_xlim(0, 240)  # outliers only
ax2.set_xlim(240, 480)  # most of the data

ax.set_ylim(0, 2500)  # outliers only


# hide the spines between ax and ax2
ax.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax.yaxis.tick_left()
ax.tick_params(labelright='off')
ax2.yaxis.tick_right()

d = .02 # how big to make the diagonal lines in axes coordinates
# arguments to pass plot, just so we don't keep repeating them
kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
ax.plot((1-d,1+d), (-d,+d), **kwargs)
ax.plot((1-d,1+d),(1-d,1+d), **kwargs)

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d,+d), (1-d,1+d), **kwargs)
ax2.plot((-d,+d), (-d,+d), **kwargs)

ax.set_xticklabels(['', '1', '2'], fontsize=14)
ax2.set_xticklabels(['7', '8', '9', '10'])

ax.tick_params(labelsize=14)
ax2.tick_params(labelsize=14)


plt.text(380, 2200, structure, fontsize=18)

plt.savefig('/home/maspe/filer/SERT/ALL/figs/' + structure + '_perispikes.pdf', dpi=150, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format='pdf', transparent=False)


In [ ]:




# NAC
iteration = 0
t = np.zeros((len(NAC_WT.keys()), window*2 / 60))
for mouse in NAC_WT.keys():
    t[iteration,:]=np.add.reduceat(NAC_WT[mouse], range(0, NAC_WT[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_WT=np.mean(t, axis=0)

iteration = 0
t = np.zeros((len(NAC_KO.keys()), window*2 / 60))
for mouse in NAC_KO.keys():
    t[iteration,:]=np.add.reduceat(NAC_KO[mouse], range(0, NAC_KO[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_KO=np.mean(t, axis=0)

plt.subplot(2,2,2)
plt.plot(tx, tmean_WT, '-', color='blue', alpha=0.7)
plt.plot(tx, tmean_KO, '-', color='red', alpha=0.7)
plt.xticks([0, 20000, 40000, 60000, 80000, 100000, 120000], ['-3', '-2', '-1', '0', '1', '2', '3'], fontsize=16)
plt.yticks(fontsize=16)
plt.ylim([0,70])
plt.axvline(x=60000, color='black')
#plt.xlabel('time (s)')
#plt.ylabel('units')
plt.title('NAC', fontsize=22)


# BLA
iteration = 0
t = np.zeros((len(BLA_WT.keys()), window*2 / 60))
for mouse in BLA_WT.keys():
    t[iteration,:]=np.add.reduceat(BLA_WT[mouse], range(0, BLA_WT[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_WT=np.mean(t, axis=0)

iteration = 0
t = np.zeros((len(BLA_KO.keys()), window*2 / 60))
for mouse in BLA_KO.keys():
    t[iteration,:]=np.add.reduceat(BLA_KO[mouse], range(0, BLA_KO[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_KO=np.mean(t, axis=0)

plt.subplot(2,2,3)
plt.plot(tx, tmean_WT, '-', color='blue', alpha=0.7)
plt.plot(tx, tmean_KO, '-', color='red', alpha=0.7)
plt.xticks([0, 20000, 40000, 60000, 80000, 100000, 120000], ['-3', '-2', '-1', '0', '1', '2', '3'], fontsize=16)
plt.yticks(fontsize=16)
plt.ylim([0,70])
plt.axvline(x=60000, color='black')
plt.xlabel('time (s)', fontsize=18)
plt.ylabel('units', fontsize=18)
plt.title('BLA', fontsize=24)


# vHip
iteration = 0
t = np.zeros((len(vHip_WT.keys()), window*2 / 60))
for mouse in vHip_WT.keys():
    t[iteration,:]=np.add.reduceat(vHip_WT[mouse], range(0, vHip_WT[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_WT=np.mean(t, axis=0)

iteration = 0
t = np.zeros((len(vHip_KO.keys()), window*2 / 60))
for mouse in vHip_KO.keys():
    t[iteration,:]=np.add.reduceat(vHip_KO[mouse], range(0, vHip_KO[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_KO=np.mean(t, axis=0)

plt.subplot(2,2,4)
plt.plot(tx, tmean_WT, '-', color='blue', alpha=0.7)
plt.plot(tx, tmean_KO, '-', color='red', alpha=0.7)
plt.xticks([0, 20000, 40000, 60000, 80000, 100000, 120000], ['-3', '-2', '-1', '0', '1', '2', '3'], fontsize=16)
plt.yticks(fontsize=16)
plt.ylim([0,70])
plt.axvline(x=60000, color='black')
plt.xlabel('time (s)', fontsize=18)
#plt.ylabel('units')
plt.title('vHip', fontsize=24)


plt.savefig('/home/maspe/filer/SERT/ALL/figs/perispikes.png', dpi=150, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format='png', transparent=False)
    
#plt.close()

#####################################

#### Backup code

In [ ]:
spikes = []
for unit in range(len(spikes_epochs_pre)):
    # np.isin(element, test_elements)
    spikes.append(np.isin(epoch_matrix, [spikes_epochs_pre[unit] + spikes_epochs_post[unit]]))

In [ ]:
spikes_epochs_pre = []
spikes_epochs_post = []
for unit in range(len(all_units)):
    spiketimes_pre = []
    spiketimes_post = []
    for epoch in range(n_epochs):
        spiketimes_pre.extend(all_units[unit][(all_units[unit] > epochs_pre[epoch][0]) & (all_units[unit] < epochs_post[epoch][1])])
        spiketimes_post.extend(all_units[unit][(all_units[unit] > epochs_post[epoch][0]) & (all_units[unit] < epochs_post[epoch][1])])
    spikes_epochs_pre.append(spiketimes_pre)
    spikes_epochs_post.append(spiketimes_post)

In [ ]:
len(spikes_epochs_pre[52])

In [ ]:
spikes = []
for unit in range(len(spikes_epochs_pre)):
    # np.isin(element, test_elements)
    spikes.append(np.isin(epoch_matrix, [spikes_epochs_pre[unit] + spikes_epochs_post[unit]]))

In [ ]:
np.sum(spikes[50], axis=0).shape

In [ ]:
### Collect the name of channels, by mouse
#all_units_WT = {}
all_channels_WT = {}

n_channels = 32
for mouse in all_mice_WT.keys():

    units = []
    channels_id = []
    
    iteration = 0
    for channel in range(n_channels):
        for unit in all_mice_WT[mouse][channel]['spiketimes'].keys():
            #units.append(all_mice_WT[mouse][channel]['spiketimes'][unit][()]) # Final "[()]" is to import values from h5py 
      
            channels_id.append(all_info_WT[mouse]['channels_locs'][iteration])
        
        iteration += 1
        
            
    #all_units_WT[mouse] = units
    all_channels_WT[mouse] = channels_id
        
# print('n units = %i' %len(all_units))

### Same for KOs
#all_units_KO = {}
all_channels_KO = {}
for mouse in all_mice_KO.keys():
    n_channels = 32
    units = []
    channels_id = []
    
    iteration = 0
    for channel in range(n_channels):
        for unit in all_mice_KO[mouse][channel]['spiketimes'].keys():
            #units.append(all_mice_KO[mouse][channel]['spiketimes'][unit][()]) # Final "[()]" is to import values from h5py 
      
            channels_id.append(all_info_KO[mouse]['channels_locs'][iteration])
        
        iteration += 1
        
            
    #all_units_KO[mouse] = units
    all_channels_KO[mouse] = channels_id
        
# print('n units = %i' %len(all_units))

